In [160]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from unidecode import unidecode
import time
from dateutil import parser

import aiohttp
import asyncio
import csv
import itertools
import os
from datetime import datetime, timedelta, date

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

pd.set_option('display.max_columns', None)

In [215]:
def ordinal(n):
    n = int(n)
    if 10 <= n % 100 < 20:
        return str(n) + 'th'
    else:
        return  str(n) + {1 : 'st', 2 : 'nd', 3 : 'rd'}.get(n % 10, "th")

def generate_fixtures(custom_fixtures = None, exclude_past_games = True):
    team_data_df = pd.read_csv('../data/team_data.csv')
    # team_data_df['gd'] = team_data_df['g_for'] - team_data_df['g_against']
    # team_data_df = team_data_df.sort_values(by='gd', ascending=False)
    team_data_dict = team_data_df.set_index('fotmob_team').T.to_dict()
    pl_team_list = team_data_dict.keys()

    competition_dict = {'pl': {'id': '47', 'ccode3': 'GBR',},
                        'fa_cup': {'id': '132', 'ccode3': 'GBR',},
                        'league_cup': {'id': '133', 'ccode3': 'GBR',},
                        'ucl': {'id': '42', 'ccode3': 'GBR',},
                        'europa_league': {'id': '73', 'ccode3': 'GBR',},
                        'europa_conference_league': {'id': '10216', 'ccode3': 'GBR',},
                        }

    match_name_list = []
    fixture_df = pd.DataFrame(columns=('comp', 'home_team', 'away_team', 'short_home_team', 'short_away_team', 'datetime_str', 'date_str', 'datetime_obj', 'fpl_gw'))

    fantasy_format = 'TFF'

    if fantasy_format == 'FPL':
        competition_subset_dict = {k:v for k, v in competition_dict.items() if k == 'pl'}
        period_separation = 'fpl_gw'
    elif fantasy_format == 'TFF':
        # need to add other formats
        competition_subset_dict = {k:v for k, v in competition_dict.items() if k == 'pl'}
        period_separation = 'datetime_str'

    # fpl_gw_starts = pd.read_csv('../data/fpl_gw_starts.csv')

    for key, params in competition_subset_dict.items():

        response = requests.get('https://www.fotmob.com/api/leagues', params=params)
        data = response.json()
        match_dict_list = data['matches']['allMatches']
        if key == 'pl':
            for match in match_dict_list:
                fotmob_home_team = match['home']['shortName']
                fotmob_away_team = match['away']['shortName']
                datetime_str = match['status']['utcTime']
                date_str = match['status']['utcTime'].split("T")[0]
                datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
                fpl_gw = match['roundName']

                short_home_team = team_data_dict[fotmob_home_team]['short_team']
                short_away_team = team_data_dict[fotmob_away_team]['short_team']
                fixture_df.loc[len(fixture_df)] = [key, fotmob_home_team, fotmob_away_team, short_home_team, short_away_team, datetime_str, date_str, datetime_obj, fpl_gw]
    fixture_df['probability'] = 1

    # add custom fixtures
    if custom_fixtures is not None:
        for index, row in custom_fixtures.iterrows():
            comp = row['comp']
            short_home_team = row['home_team']
            short_away_team = row['away_team']
            fixture_df = fixture_df.drop(fixture_df[(fixture_df['comp'] == comp) & (fixture_df['short_home_team'] == short_home_team) & (fixture_df['short_away_team'] == short_away_team)].index).reset_index(drop=True)

            for datetime_str, probability in row['datetime_probs'].items():
                date_str = datetime_str.split("T")[0]
                datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
                fotmob_home_team = team_data_df.loc[team_data_df['short_team'] == short_home_team, 'fotmob_team'].values[0]
                fotmob_away_team = team_data_df.loc[team_data_df['short_team'] == short_away_team, 'fotmob_team'].values[0]
                fixture_df.loc[len(fixture_df)] = [comp, fotmob_home_team, fotmob_away_team, short_home_team, short_away_team, datetime_str, date_str, datetime_obj, fpl_gw, probability]

    fixture_df = fixture_df.sort_values(by='datetime_obj', ascending=True)

    
    if exclude_past_games:
        today_datetime = datetime.today()
        for index, row in fixture_df.copy().iterrows():
            if row['datetime_str'] < today_datetime:
                fixture_df = fixture_df.loc[fixture_df['datetime_str']!=row['datetime_str']]
        fixture_df.reset_index(drop=True)

    num_unique_datetime = len(fixture_df[period_separation].unique())
    fixture_ticker = pd.DataFrame(columns=['team'] + ['short_team'] +list(fixture_df[period_separation].unique()))

    for team in pl_team_list:
        fixture_ticker.loc[len(fixture_ticker)] = [team] + [team_data_df.loc[team_data_df['fotmob_team']==team,'short_team'].values[0]] + [''] * num_unique_datetime

    for index, row in fixture_df.iterrows():
        prob_str = ''
        if row['probability'] != 1:
            prob_str = '*' + str(round(row['probability'],2))[1:]
        if row['home_team'] in pl_team_list:
            fixture_ticker.loc[fixture_ticker['team'] == row['home_team'], [row[period_separation]]] += '\n'+row['short_away_team'] + prob_str
        if row['away_team'] in pl_team_list:
            fixture_ticker.loc[fixture_ticker['team'] == row['away_team'], [row[period_separation]]] += '\n'+row['short_home_team'].lower() + prob_str



    # make formatted dataframe
    formatted_fixtures = fixture_ticker.copy()
    datetime_str_cols = [x for i, x in enumerate(formatted_fixtures.columns.tolist()) if len(x)>10]
    numeric_header =  [str(i-1) if 'team' not in x else x for i, x in enumerate(formatted_fixtures.columns.tolist())]
    daynum_header = ['',''] + [ordinal(parser.parse(x).strftime("%d")) for x in datetime_str_cols]
    weekday_header = ['',''] + [parser.parse(x).strftime("%a") for x in datetime_str_cols]
    gw_ref = pd.read_csv('../data/gw_ref.csv')
    tff_gw_header = ['',''] + ['GW' + str(gw_ref.loc[gw_ref['date_str']== x.split('T')[0], 'tff_gw'].values[0]) for x in datetime_str_cols]
    month_header = ['',''] + [parser.parse(x).strftime("%b") for x in datetime_str_cols]

    formatted_fixtures.columns = [month_header, tff_gw_header, weekday_header, daynum_header, numeric_header]
    # Make color map dictionary and function
    team_data_df['h_gd'] = team_data_df['g_for']*team_data_df['home_adv'] - team_data_df['g_against']/team_data_df['home_adv']
    team_data_df['a_gd'] = team_data_df['g_for']/team_data_df['home_adv'] - team_data_df['g_against']*team_data_df['home_adv']
    color_ts = team_data_df[['short_team','h_gd', 'a_gd']].copy()
    min_gd = min(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())*2.3
    max_gd = max(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())#*1.8
    norm = matplotlib.colors.Normalize(vmin=min_gd, vmax=max_gd, clip=True)
    mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis_r)
    color_ts['h_gd_color'] = color_ts['h_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    color_ts['a_gd_color'] = color_ts['a_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    h_teams = color_ts['short_team'].values.tolist()
    a_teams = [team.lower() for team in h_teams]
    teams = h_teams + a_teams
    team_gd = color_ts['a_gd_color'].values.tolist() + color_ts['h_gd_color'].values.tolist()
    color_dict = {teams[i]: team_gd[i] for i in range(len(teams))}
    def color_col(col, pattern_map, default=''):
        return np.select(
            [col.str.contains(k, na=False) for k in pattern_map.keys()],
            [f'background-color: {v}' for v in pattern_map.values()],
            default=default
        ).astype(str)
    # Apply styles
    formatted_fixtures = formatted_fixtures.style.apply(color_col,
                                                pattern_map=color_dict
                                                , subset=formatted_fixtures.columns[2:]
                                                )
    formatted_fixtures = formatted_fixtures.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)
    formatted_fixtures = formatted_fixtures.set_properties(**{'color': 'white'},subset=(formatted_fixtures.columns[2:]))

    # formatted_fixtures.columns = [formatted_fixtures.columns , [i for i, x in enumerate(formatted_fixtures.columns) ]]

    return {'formatted_fixtures': formatted_fixtures, 'unformatted_fixtures': fixture_ticker}

In [126]:
custom_fixtures = pd.DataFrame(columns=('comp', 'home_team', 'away_team', 'datetime_probs'))
# # custom_fixtures.loc[len(custom_fixtures)] = ['pl', 'MUN', 'FUL', {'2024-08-16T19:00:00Z':0.5, '2024-08-17T11:30:00Z':0.5}]
# # custom_fixtures.loc[len(custom_fixtures)] = ['pl', 'MUN', 'LIV', {'2024-08-16T19:00:00Z':0.5, '2024-08-17T12:30:00Z':0.5}]
# custom_fixtures

In [216]:
r = generate_fixtures(custom_fixtures = custom_fixtures, exclude_past_games = False)
r['formatted_fixtures']

In [193]:
r['unformatted_fixtures']

,team,short_team,2024-08-16T19:00:00Z,2024-08-17T11:30:00Z,2024-08-17T14:00:00Z,2024-08-17T16:30:00Z,2024-08-18T13:00:00Z,2024-08-18T15:30:00Z,2024-08-19T19:00:00Z,2024-08-24T11:30:00Z,2024-08-24T14:00:00Z,2024-08-24T16:30:00Z,2024-08-25T13:00:00Z,2024-08-25T15:30:00Z,2024-08-31T11:30:00Z,2024-08-31T14:00:00Z,2024-08-31T16:30:00Z,2024-09-01T12:30:00Z,2024-09-01T15:00:00Z,2024-09-14T11:30:00Z,2024-09-14T14:00:00Z,2024-09-14T16:30:00Z,2024-09-14T19:00:00Z,2024-09-15T13:00:00Z,2024-09-15T15:30:00Z,2024-09-21T11:30:00Z,2024-09-21T14:00:00Z,2024-09-21T16:30:00Z,2024-09-22T13:00:00Z,2024-09-22T15:30:00Z,2024-09-28T11:30:00Z,2024-09-28T14:00:00Z,2024-09-28T16:30:00Z,2024-09-29T13:00:00Z,2024-09-29T15:30:00Z,2024-09-30T19:00:00Z,2024-10-05T14:00:00Z,2024-10-19T14:00:00Z,2024-10-26T14:00:00Z,2024-11-02T15:00:00Z,2024-11-09T15:00:00Z,2024-11-23T15:00:00Z,2024-11-30T15:00:00Z,2024-12-03T19:45:00Z,2024-12-04T19:45:00Z,2024-12-07T15:00:00Z,2024-12-14T15:00:00Z,2024-12-21T15:00:00Z,2024-12-26T15:00:00Z,2024-12-29T15:00:00Z,2025-01-04T15:00:00Z,2025-01-14T19:45:00Z,2025-01-15T19:45:00Z,2025-01-15T20:00:00Z,2025-01-18T15:00:00Z,2025-01-25T15:00:00Z,2025-02-01T15:00:00Z,2025-02-15T15:00:00Z,2025-02-22T15:00:00Z,2025-02-25T19:45:00Z,2025-02-25T20:00:00Z,2025-02-26T19:45:00Z,2025-02-26T20:00:00Z,2025-03-08T15:00:00Z,2025-03-15T15:00:00Z,2025-04-01T18:45:00Z,2025-04-02T18:45:00Z,2025-04-02T19:00:00Z,2025-04-05T14:00:00Z,2025-04-12T14:00:00Z,2025-04-19T14:00:00Z,2025-04-26T14:00:00Z,2025-05-03T14:00:00Z,2025-05-10T14:00:00Z,2025-05-18T14:00:00Z,2025-05-25T15:00:00Z
0,Man City,MCI,,,,,,\nche,,,\nIPS,,,,,,\nwhu,,,,\nBRE,,,,,,,,,\nARS,\nnew,,,,,,\nFUL,\nwol,\nSOU,\nbou,\nbha,\nTOT,\nliv,,\nFOR,\ncry,\nMUN,\navl,\nEVE,\nlei,\nWHU,\nbre,,,\nips,\nCHE,\nars,\nNEW,\nLIV,\ntot,,,,\nfor,\nBHA,,\nLEI,,\nmun,\nCRY,\neve,\nAVL,\nWOL,\nsou,\nBOU,\nful
1,Arsenal,ARS,,,\nWOL,,,,,,,\navl,,,\nBHA,,,,,,,,,\ntot,,,,,,\nmci,,\nLEI,,,,,\nSOU,\nbou,\nLIV,\nnew,\nche,\nFOR,\nwhu,\nMUN,,\nful,\nEVE,\ncry,\nIPS,\nbre,\nbha,\nTOT,,,\nAVL,\nwol,\nMCI,\nlei,\nWHU,\nfor,,,,\nmun,\nCHE,\nFUL,,,\neve,\nBRE,\nips,\nCRY,\nBOU,\nliv,\nNEW,\nsou
2,Liverpool,LIV,,\nips,,,,,,,,,,\nBRE,,,,,\nmun,,\nFOR,,,,,,\nBOU,,,,,,\nwol,,,,\ncry,\nCHE,\nars,\nBHA,\nAVL,\nsou,\nMCI,,\nnew,\neve,\nFUL,\ntot,\nLEI,\nwhu,\nMUN,\nfor,,,\nbre,\nIPS,\nbou,\nWOL,\nmci,,,,\nNEW,\nSOU,\navl,,,\nEVE,\nful,\nWHU,\nlei,\nTOT,\nche,\nARS,\nbha,\nCRY
3,Chelsea,CHE,,,,,,\nMCI,,,,,\nwol,,,,,\nCRY,,,,,\nbou,,,\nwhu,,,,,,\nBHA,,,,,\nFOR,\nliv,\nNEW,\nmun,\nARS,\nlei,\nAVL,,\nsou,\ntot,\nBRE,\neve,\nFUL,\nips,\ncry,,\nBOU,,\nWOL,\nmci,\nWHU,\nbha,\navl,,,\nSOU,,\nLEI,\nars,,\nTOT,,\nbre,\nIPS,\nful,\nEVE,\nLIV,\nnew,\nMUN,\nfor
4,Man United,MUN,\nFUL,,,,,,,\nbha,,,,,,,,,\nLIV,\nsou,,,,,,,,\ncry,,,,,,,\nTOT,,\navl,\nBRE,\nwhu,\nCHE,\nLEI,\nips,\nEVE,\nars,,\nFOR,\nmci,\nBOU,\nwol,\nNEW,\nliv,,,\nSOU,\nBHA,\nful,\nCRY,\ntot,\neve,,,,\nIPS,\nARS,\nlei,\nfor,,,\nMCI,\nnew,\nWOL,\nbou,\nbre,\nWHU,\nche,\nAVL
5,Tottenham,TOT,,,,,,,\nlei,,\nEVE,,,,,,,\nnew,,,,,,\nARS,,,\nBRE,,,,,,,,\nmun,,\nbha,\nWHU,\ncry,\nAVL,\nIPS,\nmci,\nFUL,\nbou,,\nCHE,\nsou,\nLIV,\nfor,\nWOL,\nNEW,\nars,,,\neve,\nLEI,\nbre,\nMUN,\nips,\nMCI,,,,\nBOU,\nful,,\nche,,\nSOU,\nwol,\nFOR,\nliv,\nwhu,\nCRY,\navl,\nBHA
6,Newcastle,NEW,,,\nSOU,,,,,,,,\nbou,,,,,\nTOT,,,,,,,\nwol,,\nful,,,,\nMCI,,,,,,\neve,\nBHA,\nche,\nARS,\nfor,\nWHU,\ncry,,\nLIV,\nbre,\nLEI,\nips,\nAVL,\nmun,\ntot,,\nWOL,,\nBOU,\nsou,\nFUL,\nmci,\nFOR,,,,\nliv,\nwhu,\nCRY,,\nBRE,,\nlei,\nMUN,\navl,\nIPS,\nbha,\nCHE,\nars,\nEVE
7,Aston Villa,AVL,,,,\nwhu,,,,,,\nARS,,,,\nlei,,,,,,\nEVE,,,,,\nWOL,,,,,,,\nips,,,\nMUN,\nful,\nBOU,\ntot,\nliv,\nCRY,\nche,\nBRE,,\nSOU,\nfor,\nMCI,\nnew,\nBHA,\nLEI,\neve,,,\nars,\nWHU,\nwol,\nIPS,\nCHE,,\ncry,,,\nbre,\nLIV,\nbha,,,\nFOR,\nsou,\nNEW,\nmci,\nFUL,\nbou,\nTOT,\nmun
8,Brighton,BHA,,,\neve,,,,,\nMUN,,,,,\nars,,,,,,\nIPS,,,,,,,,\nFOR,,,\nche,,,,,\nTOT,\nnew,\nWOL,\nliv,\nMCI,\nbou,\nSOU,\nful,,\nlei,\nCRY,\nwhu,\nBRE,\navl,\nARS,\nips,,,\nmun,\nEVE,\nfor,\

In [213]:
fixture_df_copy = fixture_df.copy()
fixture_df_copy

,comp,home_team,away_team,short_home_team,short_away_team,datetime_str,date_str,datetime_obj,fpl_gw
0,pl,Man United,Fulham,MUN,FUL,2024-08-16T19:00:00Z,2024-08-16,2024-08-16 19:00:00,1
1,pl,Ipswich,Liverpool,IPS,LIV,2024-08-17T11:30:00Z,2024-08-17,2024-08-17 11:30:00,1
2,pl,Arsenal,Wolves,ARS,WOL,2024-08-17T14:00:00Z,2024-08-17,2024-08-17 14:00:00,1
3,pl,Everton,Brighton,EVE,BHA,2024-08-17T14:00:00Z,2024-08-17,2024-08-17 14:00:00,1
4,pl,Newcastle,Southampton,NEW,SOU,2024-08-17T14:00:00Z,2024-08-17,2024-08-17 14:00:00,1
...,...,...,...,...,...,...,...,...,...
374,pl,Man United,Aston Villa,MUN,AVL,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38
375,pl,Newcastle,Everton,NEW,EVE,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38
376,pl,Nottm Forest,Chelsea,FOR,CHE,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38
377,pl,Southampton,Arsenal,SOU,ARS,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38


In [210]:
today_date = datetime.today()
datetime.strftime(today_date, "%Y-%m-%dT%H:%M:%SZ")

'2024-07-28T21:06:25Z'

In [214]:
test_datetime = '2025-03-28T21:06:25Z'

today_date = datetime.today()
for index, row in fixture_df_copy.copy().iterrows():
    if row['datetime_str'] < test_datetime:
        fixture_df_copy = fixture_df_copy.loc[fixture_df_copy['datetime_str']!=row['datetime_str']]
fixture_df_copy.reset_index(drop=True)

,comp,home_team,away_team,short_home_team,short_away_team,datetime_str,date_str,datetime_obj,fpl_gw
0,pl,Bournemouth,Ipswich,BOU,IPS,2025-04-01T18:45:00Z,2025-04-01,2025-04-01 18:45:00,30
1,pl,Arsenal,Fulham,ARS,FUL,2025-04-01T18:45:00Z,2025-04-01,2025-04-01 18:45:00,30
2,pl,Brighton,Aston Villa,BHA,AVL,2025-04-01T18:45:00Z,2025-04-01,2025-04-01 18:45:00,30
3,pl,Nottm Forest,Man United,FOR,MUN,2025-04-01T18:45:00Z,2025-04-01,2025-04-01 18:45:00,30
4,pl,Wolves,West Ham,WOL,WHU,2025-04-01T18:45:00Z,2025-04-01,2025-04-01 18:45:00,30
...,...,...,...,...,...,...,...,...,...
85,pl,Man United,Aston Villa,MUN,AVL,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38
86,pl,Newcastle,Everton,NEW,EVE,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38
87,pl,Nottm Forest,Chelsea,FOR,CHE,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38
88,pl,Southampton,Arsenal,SOU,ARS,2025-05-25T15:00:00Z,2025-05-25,2025-05-25 15:00:00,38


In [175]:
fpl_gw_starts = pd.read_csv('../data/fpl_gw_starts.csv')
display(fpl_gw_starts.head())

current_date_str = '2024-08-16'
current_fpl_gw = 1
gw38_end = '2025-06-31'

# dictionary = {}
gw_ref = pd.DataFrame(columns=['date_str', 'fpl_gw'])
while current_date_str < gw38_end:
    # for index, row in fpl_gw_starts.iterrows():
    if current_date_str in fpl_gw_starts['start_date'].tolist():
        current_fpl_gw = fpl_gw_starts.loc[fpl_gw_starts['start_date']==current_date_str, 'gameweek'].values[0]

    gw_ref.loc[len(gw_ref)] = [current_date_str, current_fpl_gw]

    current_date_obj = datetime.strptime(current_date_str, '%Y-%m-%d')
    current_date_obj = current_date_obj + timedelta(days = 1)
    current_date_str = datetime.strftime(current_date_obj, "%Y-%m-%d")

# gw_ref.to_csv('../data/gw_ref.csv', index=False)

,gameweek,start_date
0,1,2024-08-16
1,2,2024-08-23
2,3,2024-08-30
3,4,2024-09-13
4,5,2024-09-20


In [182]:
tff_gw_starts = pd.read_csv('../data/tff_gw_starts.csv')
display(tff_gw_starts.head())

current_date_str = '2024-08-16'
current_tff_gw = 1
gw38_end = '2025-06-31'

# dictionary = {}
gw_ref2 = pd.DataFrame(columns=['date_str', 'tff_gw'])
while current_date_str < gw38_end:
    # for index, row in fpl_gw_starts.iterrows():
    if current_date_str in tff_gw_starts['start_date'].tolist():
        current_tff_gw = tff_gw_starts.loc[tff_gw_starts['start_date']==current_date_str, 'gameweek'].values[0]

    gw_ref2.loc[len(gw_ref2)] = [current_date_str, current_tff_gw]

    current_date_obj = datetime.strptime(current_date_str, '%Y-%m-%d')
    current_date_obj = current_date_obj + timedelta(days = 1)
    current_date_str = datetime.strftime(current_date_obj, "%Y-%m-%d")
gw_ref2

,gameweek,start_date
0,1,2024-08-16
1,2,2024-08-20
2,3,2024-08-27
3,4,2024-09-10
4,5,2024-09-17


,date_str,tff_gw
0,2024-08-16,1
1,2024-08-17,1
2,2024-08-18,1
3,2024-08-19,1
4,2024-08-20,2
...,...,...
314,2025-06-26,37
315,2025-06-27,37
316,2025-06-28,37
317,2025-06-29,37


In [183]:
merged_gw_ref = pd.merge(left=gw_ref, right=gw_ref2, on='date_str')
merged_gw_ref.to_csv('../data/gw_ref.csv', index=False)